<a href="https://colab.research.google.com/github/gabriellfelipe7/Alura_EstatisticaComPython/blob/main/Ex2_SCC0634_2022_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exercício 2 - SCC0634: Aplicações de Inteligência Artificial (2022/02)**
- Nome: Gabriel Felipe Machado de Oliveira
- N° USP: 11908695

In [ ]:
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# Extração da Base

Através da bib Pandas, importaremos a base:

In [ ]:
data = pd.read_csv('Dmoz-Sports.csv')
display(data)

,Text,Class
0,articl galleri fan match report pictur site forum,Soccer
1,bodi fan fox profit interest support citi demo...,Soccer
2,club burnlei literatur team onlin form photograph,Soccer
3,club histori inform honour wikipedia record to...,Soccer
4,uefa uefa footbal footbal championship champio...,Soccer
...,...,...
13495,press golden california staff bear releas berk...,Tennis
13496,northwood staff game men tenni pictur seahawk ...,Tennis
13497,player club england pro harri jason associ ten...,Tennis
13498,stand individu inform men team tenni pictur re...,Tennis


# Lista de Classes

Vamos verificar a lista de possíveis saídas da variável resposta:

In [ ]:
esportes = list(data['Class'].unique())
esportes

['Soccer',
 'Fencing',
 'Equestrian',
 'Wrestling',
 'Volleyball',
 'Winter_Sports',
 'Basketball',
 'Lacrosse',
 'Football',
 'Martial_Arts',
 'Softball',
 'Paintball',
 'Flying_Discs',
 'Cycling',
 'Running',
 'Baseball',
 'Cricket',
 'Gymnastics',
 'Motorsports',
 'Golf',
 'Skating',
 'Track_and_Field',
 'Water_Sports',
 'Bowling',
 'Hockey',
 'Strength_Sports',
 'Tennis']

In [ ]:
len(esportes)

27

Temos 27 possíveis saídas para os dados.

# Separando base de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(data, test_size=0.33)

Verificando os dados:

In [ ]:
df_test.shape

(4455, 2)

In [ ]:
df_train.shape

(9045, 2)

# Métodos de apoio

In [ ]:
def remove_stopwords(text,stop_words):
  
  # tudo para caixa baixa
  s = str(text).lower() 

  tokens = word_tokenize(s)

  # remove stopwords, dígitos, caracteres especiais e pontuações
  v = [word for word in tokens if not word in stop_words and word.isalnum() and not word.isdigit()]

  return v

def stemming(tokens,stemmer):
  tokens_stems = [stemmer.stem(word) for word in tokens]
  return tokens_stems

def meu_tokenizador(doc, stop_words=nltk.corpus.stopwords.words('english'), stemmer=PorterStemmer()):
  tokens = remove_stopwords(doc,stop_words)
  return stemming(tokens,stemmer)

#Pré-processamento e Treinamento
Utilizamos o o modelo de transformação geral "distiluse-base-multilingual-cased-v1"

In [ ]:
! pip install sentence_transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Vamos carregar o modelo
sbert  = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
#Separando as palavras das classes e pre ptrocessão o encoder


documents = np.array(df_train['Text'].to_list())
labels = np.array(df_train['Class'].to_list())


# vamos usar validação cruzada estratificada com 5 folds
skf = StratifiedKFold(n_splits=2, random_state=42, shuffle=True)

for k in [5,7]:
  fold = 1
  L_acc = []
  for train_index, test_index in skf.split(documents, labels):

      # separando os documentos de treino e teste considerando cada fold.
      documents_train, documents_test = documents[train_index], documents[test_index]
      labels_train, labels_test = labels[train_index], labels[test_index]

      # preprocessamento
      sbert  = SentenceTransformer('distiluse-base-multilingual-cased-v1')
      X_train2 = sbert.encode(documents_train)
      X_test2 = sbert.encode(documents_test)

      # classificador 1NN do Cenario 2
      clf2 = KNeighborsClassifier(n_neighbors=k,metric='cosine')
      clf2.fit(X_train2,labels_train)

      # imprimindo resultados
      print('Fold =',fold,' k=',k)
      acc = clf2.score(X_test2, labels_test)
      print('\tACC usando TF-IDF =',acc)

      L_acc.append(acc)

      fold += 1
  print('acc k=',k,np.mean(L_acc))
  print('=======')

Fold = 1  k= 5
	ACC usando TF-IDF = 0.6933451249170904
Fold = 2  k= 5
	ACC usando TF-IDF = 0.7043343653250774
acc k= 5 0.6988397451210839
Fold = 1  k= 7
	ACC usando TF-IDF = 0.698872429803228
Fold = 2  k= 7
	ACC usando TF-IDF = 0.7091994692613888
acc k= 7 0.7040359495323083


ERRATA: Perceba que foi utilizado o outro modo de codificação, e não TF-IDF, por conta da demora de execução, foi impossível a executar novamente a célula.

# Decisão

Usaremos k = 7

In [ ]:
k=7

# separando os documentos de treino e teste considerando cada fold.

# preprocessamento (ponderação TF-IDF)
sbert  = SentenceTransformer('distiluse-base-multilingual-cased-v1')
X_train2 = sbert.encode(df_train['Text'].to_list())
X_test2 = sbert.encode(df_test['Text'].to_list())

# classificador 1NN do Cenario 2
clf2 = KNeighborsClassifier(n_neighbors=k,metric='cosine')
clf2.fit(X_train2,df_train['Class'].to_list())

# imprimindo resultados
acc = clf2.score(X_test2, df_test['Class'].to_list())
print('\tACC usando Sentence Transformer =',acc)

	ACC usando Sentence Transformer = 0.7441077441077442


Utilizando kFoldValidation e a Sentence Transformer, alcançamos uma acurácia (medida cosseno) de 74.41%.